In [9]:
from pykeen.datasets import FB15k237
from pykeen.models import TransE
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation import RankBasedEvaluator

from torch.optim import Adam

import pandas as pd

In [3]:

dataset = FB15k237()
training_triples_factory = dataset.training

model = TransE(triples_factory=training_triples_factory)

optim = Adam(model.get_grad_params())



You're trying to map triples with 30 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 28 from 20466 triples were filtered out
No random seed is specified. This may lead to non-reproducible results.


In [11]:
training_pykeen = dataset.training.triples
df = pd.read_table('./data/train.txt', header=None, names=['subject','predicate','object'])


Reconstructing all label-based triples. This is expensive and rarely needed.


In [18]:
print(sorted(df['subject'].sort_values().values) == sorted(training_pykeen[:,0]))
print(sorted(df['object'].sort_values().values) == sorted(training_pykeen[:,2]))


True
True


In [ ]:
training_loop = SLCWATrainingLoop(
    model=model,
    triples_factory=training_triples_factory,
    optimizer=optim
)

_ = training_loop.train(
    triples_factory=training_triples_factory,
    num_epochs=5,
    batch_size=256
)

evaluator = RankBasedEvaluator()

mapped_triples = dataset.testing.mapped_triples

results = evaluator.evaluate(
    model=model,
    mapped_triples=mapped_triples,
    batch_size=1024,
    additional_filter_triples = [
        dataset.testing.mapped_triples,
        dataset.validation.mapped_triples
    ],
)